<a href="https://colab.research.google.com/github/aymansalem/AymanSalem.github.io/blob/main/wan2gp-google-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wan2GP on Google Colab

Sets up [Wan2GP](https://github.com/deepbeepmeep/Wan2GP) in a fresh GPU-backed Colab session.

Run the cells in order to prepare the runtime, install dependencies, and launch the Gradio interface. Click on the link in the output from the last cell to launch the app in your browser.

> **Colab VRAM note:** the free tier usually assigns a 15 GB T4 GPU. Most Wan2GP models exceed that budget; the Wan 2.2 TextImage2Video FastWan model works, producing roughly a 5 second 480p clip in about 8 minutes.


📋 CELL 1: Environment Check & GPU Setup


In [7]:
"""
═══════════════════════════════════════════════════════════════════════════════
WAN2GP ON GOOGLE COLAB - COMPLETE SETUP
═══════════════════════════════════════════════════════════════════════════════

CELL 1 OF 6: ENVIRONMENT CHECK & GPU SETUP

What this cell does:
- Verifies GPU is available
- Checks GPU memory
- Sets up workspace directories
- Validates Python version

Run time: ~30 seconds
═══════════════════════════════════════════════════════════════════════════════
"""

import subprocess
import sys
from pathlib import Path

print("=" * 70)
print("CELL 1: ENVIRONMENT CHECK & GPU SETUP")
print("=" * 70)

# Step 1: Verify GPU is available
print("\n[1/3] Checking GPU availability...")
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
        capture_output=True,
        text=True,
        check=True
    )
    gpu_info = result.stdout.strip()
    print(f"✅ GPU Detected: {gpu_info}")

    # Check memory
    memory_result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,noheader,nounits'],
        capture_output=True,
        text=True,
        check=True
    )
    free_memory = int(memory_result.stdout.strip())

    if free_memory < 8000:
        print(f"⚠️  Warning: Only {free_memory}MB GPU memory available. Recommended: 8GB+")
    else:
        print(f"✅ GPU Memory: {free_memory}MB available")

except subprocess.CalledProcessError:
    print("\n❌ ERROR: GPU not detected!")
    print("\nFIX THIS:")
    print("1. Click 'Runtime' in the menu above")
    print("2. Select 'Change runtime type'")
    print("3. Choose 'T4 GPU' under Hardware accelerator")
    print("4. Click 'Save'")
    print("5. Re-run this cell")
    sys.exit(1)
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    sys.exit(1)

# Step 2: Configure workspace
print("\n[2/3] Configuring workspace...")
WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
MODEL_CACHE = WAN2GP_ROOT / 'models'
print(f"✅ Workspace: {WAN2GP_ROOT}")
print(f"✅ Model cache: {MODEL_CACHE}")

# Step 3: Check Python version
print("\n[3/3] Checking Python version...")
python_version = sys.version.split()[0]
print(f"✅ Python version: {python_version}")

if sys.version_info < (3, 8):
    print("⚠️  Warning: Python 3.8+ recommended")

print("\n" + "=" * 70)
print("✅ ENVIRONMENT CHECK COMPLETE")
print("=" * 70)
print("\n➡️  Next: Run CELL 2 to install system dependencies")

CELL 1: ENVIRONMENT CHECK & GPU SETUP

[1/3] Checking GPU availability...
✅ GPU Detected: Tesla T4, 15360 MiB
✅ GPU Memory: 15095MB available

[2/3] Configuring workspace...
✅ Workspace: /content/Wan2GP
✅ Model cache: /content/Wan2GP/models

[3/3] Checking Python version...
✅ Python version: 3.12.12

✅ ENVIRONMENT CHECK COMPLETE

➡️  Next: Run CELL 2 to install system dependencies


📋 CELL 2: Install System Dependencies


In [8]:
"""
═══════════════════════════════════════════════════════════════════════════════
CELL 2 OF 6: INSTALL SYSTEM DEPENDENCIES

What this cell does:
- Updates package lists
- Installs ffmpeg (video processing)
- Installs system libraries (OpenGL, audio)
- Upgrades pip

Run time: ~1-2 minutes
═══════════════════════════════════════════════════════════════════════════════
"""

import subprocess
import os
import sys

print("=" * 70)
print("CELL 2: INSTALLING SYSTEM DEPENDENCIES")
print("=" * 70)

# Configure non-interactive mode to prevent prompts
env = os.environ.copy()
env['DEBIAN_FRONTEND'] = 'noninteractive'

def run_command(cmd, description, env=None):
    """Run a command with error handling and progress indication"""
    print(f"\n{description}...")
    try:
        result = subprocess.run(
            cmd,
            env=env,
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✅ {description} - Complete")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {description} - Failed")
        print(f"Error: {e.stderr}")
        return False

# Step 1: Update package lists
print("\n[1/4] Updating package lists...")
if not run_command(
    ['sudo', 'apt-get', 'update', '-qq'],
    "Package list update",
    env=env
):
    print("⚠️  Warning: Package update failed. Continuing anyway...")

# Step 2: Install system packages
print("\n[2/4] Installing system packages...")
packages = [
    'ffmpeg',           # Video processing
    'libglib2.0-0',     # Core system library
    'libgl1',           # OpenGL support
    'libportaudio2',    # Audio support
]

success = run_command(
    ['sudo', 'apt-get', 'install', '-y', '--no-install-recommends'] + packages,
    "System packages installation",
    env=env
)

if not success:
    print("\n❌ CRITICAL ERROR: System packages installation failed")
    print("This may cause issues later. Try running this cell again.")
    sys.exit(1)

# Step 3: Verify installations
print("\n[3/4] Verifying installations...")
try:
    ffmpeg_version = subprocess.run(
        ['ffmpeg', '-version'],
        capture_output=True,
        text=True,
        check=True
    )
    version_line = ffmpeg_version.stdout.split('\n')[0]
    print(f"✅ {version_line}")
except Exception as e:
    print(f"⚠️  FFmpeg verification failed: {e}")

# Step 4: Upgrade pip
print("\n[4/4] Upgrading pip, setuptools, and wheel...")
success = run_command(
    [sys.executable, '-m', 'pip', 'install', '--upgrade', '--quiet', 'pip', 'setuptools', 'wheel'],
    "Pip upgrade"
)

if not success:
    print("⚠️  Warning: Pip upgrade failed. Continuing anyway...")

print("\n" + "=" * 70)
print("✅ SYSTEM DEPENDENCIES INSTALLED")
print("=" * 70)
print("\n➡️  Next: Run CELL 3 to clone/update Wan2GP repository")

CELL 2: INSTALLING SYSTEM DEPENDENCIES

[1/4] Updating package lists...

Package list update...
✅ Package list update - Complete

[2/4] Installing system packages...

System packages installation...
✅ System packages installation - Complete

[3/4] Verifying installations...
✅ ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers

[4/4] Upgrading pip, setuptools, and wheel...

Pip upgrade...
✅ Pip upgrade - Complete

✅ SYSTEM DEPENDENCIES INSTALLED

➡️  Next: Run CELL 3 to clone/update Wan2GP repository


📋 CELL 3: Clone/Update Repository


In [9]:
"""
═══════════════════════════════════════════════════════════════════════════════
CELL 3 OF 6: CLONE/UPDATE WAN2GP REPOSITORY

What this cell does:
- Downloads Wan2GP from GitHub (or updates if exists)
- Verifies repository structure
- Shows current git commit

Run time: ~2-3 minutes (first time)
          ~30 seconds (updates)
═══════════════════════════════════════════════════════════════════════════════
"""

import subprocess
import sys
from pathlib import Path

print("=" * 70)
print("CELL 3: REPOSITORY SETUP")
print("=" * 70)

# Configuration
WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
REPO_URL = 'https://github.com/deepbeepmeep/Wan2GP.git'

print(f"\nRepository: {REPO_URL}")
print(f"Target directory: {WAN2GP_ROOT}")

def run_git_command(cmd, description):
    """Run a git command with error handling"""
    print(f"\n{description}...")
    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✅ {description} - Complete")
        if result.stdout:
            print(result.stdout.strip())
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {description} - Failed")
        print(f"Error output: {e.stderr}")
        return False

# Check if repository already exists
if WAN2GP_ROOT.exists():
    print("\n[1/2] Repository found. Updating to latest version...")

    # Verify it's actually a git repository
    if (WAN2GP_ROOT / '.git').exists():
        success = run_git_command(
            ['git', '-C', str(WAN2GP_ROOT), 'pull', '--rebase'],
            "Git pull"
        )

        if not success:
            print("\n⚠️  Update failed. Attempting to reset repository...")
            run_git_command(
                ['git', '-C', str(WAN2GP_ROOT), 'reset', '--hard', 'HEAD'],
                "Git reset"
            )
            run_git_command(
                ['git', '-C', str(WAN2GP_ROOT), 'pull', '--rebase'],
                "Git pull (retry)"
            )
    else:
        print("⚠️  Directory exists but is not a git repository. Removing...")
        import shutil
        shutil.rmtree(WAN2GP_ROOT)
        print("✅ Old directory removed. Will clone fresh copy...")

# Clone if doesn't exist
if not WAN2GP_ROOT.exists():
    print("\n[1/2] Cloning repository (this may take 2-3 minutes)...")
    success = run_git_command(
        ['git', 'clone', '--depth', '1', REPO_URL, str(WAN2GP_ROOT)],
        "Git clone"
    )

    if not success:
        print("\n❌ CRITICAL ERROR: Failed to clone repository")
        print("\nPossible causes:")
        print("1. Network connectivity issues")
        print("2. Repository URL changed or unavailable")
        print("3. GitHub API rate limit reached")
        print("\nTry running this cell again in a few minutes.")
        sys.exit(1)

# Verify repository contents
print("\n[2/2] Verifying repository structure...")
required_files = [
    'wgp.py',
    'requirements.txt',
    'preprocessing',
]

missing_files = []
for item in required_files:
    path = WAN2GP_ROOT / item
    if not path.exists():
        missing_files.append(item)
        print(f"❌ Missing: {item}")
    else:
        print(f"✅ Found: {item}")

if missing_files:
    print("\n⚠️  WARNING: Some required files are missing!")
    print("The repository may be incomplete. Consider deleting /content/Wan2GP")
    print("and running this cell again.")
else:
    print("✅ All required files present")

# Show git commit info
try:
    commit_info = subprocess.run(
        ['git', '-C', str(WAN2GP_ROOT), 'log', '-1', '--oneline'],
        capture_output=True,
        text=True,
        check=True
    )
    print(f"\n📌 Current commit: {commit_info.stdout.strip()}")
except:
    pass

print("\n" + "=" * 70)
print("✅ REPOSITORY SETUP COMPLETE")
print("=" * 70)
print("\n➡️  Next: Run CELL 4 to install Python dependencies")

CELL 3: REPOSITORY SETUP

Repository: https://github.com/deepbeepmeep/Wan2GP.git
Target directory: /content/Wan2GP

[1/2] Repository found. Updating to latest version...

Git pull...
❌ Git pull - Failed
Error output: error: cannot pull with rebase: You have unstaged changes.
error: please commit or stash them.


⚠️  Update failed. Attempting to reset repository...

Git reset...
✅ Git reset - Complete
HEAD is now at b566924 Add FlashVSR plugin for AI video upscaling

Git pull (retry)...
✅ Git pull (retry) - Complete
Already up to date.

[2/2] Verifying repository structure...
✅ Found: wgp.py
✅ Found: requirements.txt
✅ Found: preprocessing
✅ All required files present

📌 Current commit: b566924 Add FlashVSR plugin for AI video upscaling

✅ REPOSITORY SETUP COMPLETE

➡️  Next: Run CELL 4 to install Python dependencies


📋 CELL 4: Install Python Dependencies


In [10]:
"""
═══════════════════════════════════════════════════════════════════════════════
CELL 4 OF 6: INSTALL PYTHON DEPENDENCIES

What this cell does:
- Installs PyTorch with CUDA support
- Installs xFormers (memory optimization)
- Installs Wan2GP requirements
- Applies Colab compatibility patches

Run time: ~5-10 minutes (first time)
          ~2-3 minutes (subsequent runs if cached)

This is the longest cell - please be patient!
═══════════════════════════════════════════════════════════════════════════════
"""

import subprocess
import sys
from pathlib import Path
import time

print("=" * 70)
print("CELL 4: INSTALLING PYTHON DEPENDENCIES")
print("=" * 70)
print("\n⏱️  Estimated time: 5-10 minutes (first run)")
print("    Subsequent runs will be faster if packages are cached.\n")

WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
start_time = time.time()

def install_package(cmd, description, critical=True):
    """Install a package with progress indication"""
    print(f"\n{'=' * 70}")
    print(f"{description}")
    print('=' * 70)

    try:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )

        # Show real-time output for long installations
        for line in process.stdout:
            line = line.strip()
            if line and not line.startswith('  '):  # Filter noise
                print(line)

        process.wait()

        if process.returncode == 0:
            print(f"✅ {description} - Complete")
            return True
        else:
            print(f"❌ {description} - Failed (exit code: {process.returncode})")
            return False

    except Exception as e:
        print(f"❌ {description} - Exception: {e}")
        return False

# Step 1: Install PyTorch (CUDA 12.1 for Colab T4)
print("\n[1/4] Installing PyTorch with CUDA support...")
print("This is the largest package (~2GB). Please wait...")

success = install_package(
    [sys.executable, '-m', 'pip', 'install', '--upgrade',
     'torch', 'torchvision', 'torchaudio', '--index-url',
     'https://download.pytorch.org/whl/cu121'],
    "PyTorch + CUDA 12.1"
)

if not success:
    print("\n❌ CRITICAL ERROR: PyTorch installation failed")
    print("Wan2GP cannot run without PyTorch. Try running this cell again.")
    sys.exit(1)

# Verify PyTorch installation
print("\nVerifying PyTorch installation...")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ CUDA version: {torch.version.cuda}")
        print(f"✅ GPU device: {torch.cuda.get_device_name(0)}")
except Exception as e:
    print(f"⚠️  PyTorch verification failed: {e}")

# Step 2: Install xFormers (memory-efficient attention)
print("\n[2/4] Installing xFormers (optimized attention mechanisms)...")

success = install_package(
    [sys.executable, '-m', 'pip', 'install', 'xformers',
     '--index-url', 'https://download.pytorch.org/whl/cu121'],
    "xFormers"
)

if not success:
    print("⚠️  xFormers installation failed. This may reduce performance but is not critical.")

# Step 3: Install Wan2GP requirements
print("\n[3/4] Installing Wan2GP requirements...")

requirements_file = WAN2GP_ROOT / 'requirements.txt'

if not requirements_file.exists():
    print(f"❌ ERROR: requirements.txt not found at {requirements_file}")
    print("Make sure Cell 3 (Repository Setup) completed successfully.")
    sys.exit(1)

# Show what we're about to install
print(f"\nReading requirements from: {requirements_file}")
try:
    with open(requirements_file) as f:
        req_content = f.read().strip()
        req_lines = [line for line in req_content.split('\n')
                     if line and not line.startswith('#')]
        print(f"Found {len(req_lines)} packages to install\n")
except Exception as e:
    print(f"⚠️  Could not read requirements: {e}")

success = install_package(
    [sys.executable, '-m', 'pip', 'install', '-r', str(requirements_file)],
    "Wan2GP requirements"
)

if not success:
    print("\n⚠️  Some requirements may have failed to install.")
    print("The application might still work. Continue to next cell.")

# Step 4: Apply Colab-specific patches
print("\n[4/4] Applying Colab compatibility patches...")

# Patch matplotlib to use headless backend
interact_tools = WAN2GP_ROOT / 'preprocessing/matanyone/tools/interact_tools.py'

if interact_tools.exists():
    try:
        content = interact_tools.read_text()
        if "matplotlib.use('TkAgg')" in content:
            content = content.replace("matplotlib.use('TkAgg')", "matplotlib.use('Agg')")
            interact_tools.write_text(content)
            print("✅ Matplotlib patched for headless mode")
        else:
            print("✅ Matplotlib already configured correctly")
    except Exception as e:
        print(f"⚠️  Failed to patch matplotlib: {e}")
        print("This may cause display issues but is not critical.")
else:
    print("ℹ️  interact_tools.py not found - patch not needed")

# Installation complete
elapsed = time.time() - start_time
minutes = int(elapsed // 60)
seconds = int(elapsed % 60)

print("\n" + "=" * 70)
print("✅ ALL DEPENDENCIES INSTALLED")
print("=" * 70)
print(f"\n⏱️  Total installation time: {minutes}m {seconds}s")
print("\n➡️  Next: Run CELL 5 to launch Wan2GP interface")
print("        OR run CELL 6 to launch with Google Drive integration")

CELL 4: INSTALLING PYTHON DEPENDENCIES

⏱️  Estimated time: 5-10 minutes (first run)
    Subsequent runs will be faster if packages are cached.


[1/4] Installing PyTorch with CUDA support...
This is the largest package (~2GB). Please wait...

PyTorch + CUDA 12.1
Looking in indexes: https://download.pytorch.org/whl/cu121
✅ PyTorch + CUDA 12.1 - Complete

Verifying PyTorch installation...
✅ PyTorch version: 2.8.0+cu128
✅ CUDA available: True
✅ CUDA version: 12.8
✅ GPU device: Tesla T4

[2/4] Installing xFormers (optimized attention mechanisms)...

xFormers
Looking in indexes: https://download.pytorch.org/whl/cu121
✅ xFormers - Complete

[3/4] Installing Wan2GP requirements...

Reading requirements from: /content/Wan2GP/requirements.txt
Found 57 packages to install


Wan2GP requirements
Ignoring insightface: markers 'sys_platform == "win32" and python_version == "3.10"' don't match your environment
✅ Wan2GP requirements - Complete

[4/4] Applying Colab compatibility patches...
✅ Matplotl

📋 CELL 5: Launch Wan2GP Interface


In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════════
CELL 5 OF 6: LAUNCH WAN2GP INTERFACE (Standard)

What this cell does:
- Launches the Gradio web interface
- Auto-selects appropriate model based on GPU
- Creates public URL for access
- Keeps session alive

INSTRUCTIONS:
1. Run this cell
2. Wait for "Running on public URL: https://..."
3. Click the URL to open interface
4. Keep this cell running while using Wan2GP
5. Press STOP button when done

Run time: ~10-15 minutes (first launch - downloads models)
          ~2-3 minutes (subsequent launches)

NOTE: Use CELL 6 instead if you want to save outputs to Google Drive
═══════════════════════════════════════════════════════════════════════════════
"""

import os
import sys
import subprocess
import threading
import time
from pathlib import Path

print("=" * 70)
print("CELL 5: LAUNCHING WAN2GP")
print("=" * 70)

# Configuration
WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
MODEL_CACHE = WAN2GP_ROOT / 'models'

# Detect GPU type to recommend profile
def get_recommended_profile():
    """Detect GPU and recommend appropriate model profile"""
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'],
            capture_output=True,
            text=True,
            check=True
        )
        gpu_name = result.stdout.strip().lower()

        if 'a100' in gpu_name or 'v100' in gpu_name:
            return '14', 'A100/V100 detected - using 14B model (high quality)'
        else:
            return '1', 'T4 detected - using 1.3B model (optimized for free tier)'
    except:
        return '1', 'GPU detection failed - using 1.3B model (safe default)'

PROFILE, profile_reason = get_recommended_profile()

print(f"\n🎯 Auto-selected Profile: {PROFILE}")
print(f"   Reason: {profile_reason}")
print("\nTo change profile, modify the PROFILE variable in this cell:")
print("  PROFILE = '1'   # For 1.3B model (T4 GPU)")
print("  PROFILE = '14'  # For 14B model (A100 GPU)")

# Validate workspace
if not WAN2GP_ROOT.exists():
    print("\n❌ ERROR: Wan2GP directory not found!")
    print("Please run Cell 3 (Repository Setup) first.")
    sys.exit(1)

wgp_script = WAN2GP_ROOT / 'wgp.py'
if not wgp_script.exists():
    print(f"\n❌ ERROR: wgp.py not found at {wgp_script}")
    print("Repository may be corrupted. Re-run Cell 3.")
    sys.exit(1)

# Setup environment
print("\n[1/3] Configuring environment...")
env = os.environ.copy()
env['WAN_CACHE_DIR'] = str(MODEL_CACHE)
env['PYTHONUNBUFFERED'] = '1'  # Ensure real-time output

print(f"✅ Model cache directory: {MODEL_CACHE}")
print(f"✅ Working directory: {WAN2GP_ROOT}")

# Prepare launch command
cmd = [
    sys.executable, '-u', 'wgp.py',
    '--listen',                    # Listen on all interfaces
    '--server-port', '7860',       # Gradio default port
    '--share',                     # Create public URL
    '--profile', PROFILE           # Model profile
]

print(f"\n[2/3] Launch command:")
print(f"     {' '.join(cmd)}")

# Keep-alive thread to prevent Colab timeout
stop_event = threading.Event()
keepalive_interval = 45  # seconds

def keepalive_thread():
    """Keep Colab session alive by printing periodic messages"""
    counter = 0
    while not stop_event.is_set():
        time.sleep(keepalive_interval)
        if not stop_event.is_set():
            counter += 1
            elapsed_min = (counter * keepalive_interval) // 60
            print(f"\n[Keep-alive] Session active - {elapsed_min} minutes running...")

print("\n[3/3] Starting Wan2GP...")
print("\n" + "=" * 70)
print("🚀 LAUNCHING APPLICATION")
print("=" * 70)
print("\n⏳ First launch downloads models (~5GB, 10-15 minutes)")
print("📡 Wait for the Gradio URL to appear below...")
print("🔗 Look for: 'Running on public URL: https://...'")
print("\n" + "-" * 70 + "\n")

# Start keep-alive thread
keepalive = threading.Thread(target=keepalive_thread, daemon=True)
keepalive.start()

# Launch the application
try:
    process = subprocess.Popen(
        cmd,
        cwd=str(WAN2GP_ROOT),
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    # Stream output in real-time
    gradio_url_found = False

    for line in iter(process.stdout.readline, ''):
        if not line:
            break

        # Print the line
        print(line, end='')

        # Highlight the Gradio URL
        if 'Running on public URL:' in line and not gradio_url_found:
            gradio_url_found = True
            print("\n" + "=" * 70)
            print("✅ WAN2GP IS READY!")
            print("=" * 70)
            print("👆 Click the URL above to open the interface")
            print("⏹️  Press the STOP button in Colab when done")
            print("=" * 70 + "\n")

    # Process ended
    process.wait()
    exit_code = process.returncode

    if exit_code != 0:
        print(f"\n⚠️  Process exited with code {exit_code}")
    else:
        print("\n✅ Wan2GP shut down normally")

except KeyboardInterrupt:
    print("\n\n⏹️  Stopping Wan2GP...")
    process.terminate()
    try:
        process.wait(timeout=5)
    except subprocess.TimeoutExpired:
        print("⚠️  Force killing process...")
        process.kill()
    print("✅ Wan2GP stopped")

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    print("\nTroubleshooting:")
    print("1. Check that all previous cells completed successfully")
    print("2. Verify GPU is still available (re-run Cell 1)")
    print("3. Try restarting runtime: Runtime → Restart runtime")

finally:
    # Clean up
    stop_event.set()
    keepalive.join(timeout=1)

    print("\n" + "=" * 70)
    print("SESSION ENDED")
    print("=" * 70)
    print("\nTo restart Wan2GP, simply re-run this cell.")

CELL 5: LAUNCHING WAN2GP

🎯 Auto-selected Profile: 1
   Reason: T4 detected - using 1.3B model (optimized for free tier)

To change profile, modify the PROFILE variable in this cell:
  PROFILE = '1'   # For 1.3B model (T4 GPU)
  PROFILE = '14'  # For 14B model (A100 GPU)

[1/3] Configuring environment...
✅ Model cache directory: /content/Wan2GP/models
✅ Working directory: /content/Wan2GP

[2/3] Launch command:
     /usr/bin/python3 -u wgp.py --listen --server-port 7860 --share --profile 1

[3/3] Starting Wan2GP...

🚀 LAUNCHING APPLICATION

⏳ First launch downloads models (~5GB, 10-15 minutes)
📡 Wait for the Gradio URL to appear below...
🔗 Look for: 'Running on public URL: https://...'

----------------------------------------------------------------------

2025-12-25 10:29:17.893351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766658557.

📋 CELL 6: Launch with Google Drive (OPTIONAL)


In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════════
CELL 6 OF 6: LAUNCH WAN2GP WITH GOOGLE DRIVE (Optional Alternative to Cell 5)

What this cell does:
- Everything from Cell 5 PLUS:
- Mounts Google Drive
- Saves all videos to Drive automatically
- Preserves outputs after session ends

USE THIS CELL IF:
- You want videos saved permanently
- You need to access videos from other devices
- Colab session might disconnect

OUTPUT LOCATION: /MyDrive/Wan2GP_Videos/

INSTRUCTIONS:
1. Run this cell (DON'T run Cell 5 if using this)
2. Authorize Google Drive access when prompted
3. Wait for Gradio URL
4. Click URL to use interface
5. Videos auto-save to Drive

Run time: ~10-15 minutes (first launch)
          + 1-2 minutes for Drive setup
═══════════════════════════════════════════════════════════════════════════════
"""

import os
import sys
import subprocess
import threading
import time
from pathlib import Path

print("=" * 70)
print("CELL 6: LAUNCHING WAN2GP WITH GOOGLE DRIVE")
print("=" * 70)

# Step 1: Mount Google Drive
print("\n[1/5] Connecting to Google Drive...")

try:
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        print("📁 Requesting Drive access...")
        print("   → Click the link below")
        print("   → Sign in to your Google account")
        print("   → Click 'Allow' to grant access\n")
        drive.mount('/content/drive')
        print("\n✅ Google Drive connected!")
    else:
        print("✅ Google Drive already connected")

except Exception as e:
    print(f"❌ ERROR: Could not connect to Google Drive")
    print(f"   Error: {e}")
    print("\nPossible solutions:")
    print("1. Re-run this cell and complete the authentication")
    print("2. Use Cell 5 instead (no Drive integration)")
    sys.exit(1)

# Step 2: Setup directories
print("\n[2/5] Setting up directories...")

WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
MODEL_CACHE = WAN2GP_ROOT / 'models'
DRIVE_OUTPUT = Path('/content/drive/MyDrive/Wan2GP_Videos')

# Create output directory in Drive
try:
    DRIVE_OUTPUT.mkdir(parents=True, exist_ok=True)
    print(f"✅ Output directory: {DRIVE_OUTPUT}")

    # Create a README in the folder
    readme = DRIVE_OUTPUT / 'README.txt'
    if not readme.exists():
        readme.write_text(
            "Wan2GP Video Outputs\n"
            "====================\n\n"
            "This folder contains videos generated by Wan2GP in Google Colab.\n"
            f"Created: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n"
            "Videos are organized by generation timestamp.\n"
        )

    # Test write permissions
    test_file = DRIVE_OUTPUT / '.test_write'
    test_file.write_text('test')
    test_file.unlink()
    print("✅ Write permissions confirmed")

except Exception as e:
    print(f"❌ ERROR: Cannot create output directory")
    print(f"   Error: {e}")
    print("\nCheck that you have sufficient Drive storage space.")
    sys.exit(1)

# Step 3: Detect GPU and select profile
print("\n[3/5] Detecting GPU...")

def get_recommended_profile():
    """Detect GPU and recommend appropriate model profile"""
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'],
            capture_output=True,
            text=True,
            check=True
        )
        gpu_name = result.stdout.strip().lower()

        if 'a100' in gpu_name or 'v100' in gpu_name:
            return '14', 'A100/V100 detected - using 14B model'
        else:
            return '1', 'T4 detected - using 1.3B model'
    except:
        return '1', 'Detection failed - using 1.3B model (safe default)'

PROFILE, profile_reason = get_recommended_profile()
print(f"✅ Profile selected: {PROFILE}")
print(f"   {profile_reason}")

# Step 4: Configure environment
print("\n[4/5] Configuring environment...")

if not WAN2GP_ROOT.exists() or not (WAN2GP_ROOT / 'wgp.py').exists():
    print("\n❌ ERROR: Wan2GP not found!")
    print("Please run Cells 1-4 first to install Wan2GP.")
    sys.exit(1)

env = os.environ.copy()
env['WAN_CACHE_DIR'] = str(MODEL_CACHE)
env['WAN2GP_OUTPUT_DIR'] = str(DRIVE_OUTPUT)  # Custom output location
env['PYTHONUNBUFFERED'] = '1'

print(f"✅ Model cache: {MODEL_CACHE}")
print(f"✅ Output directory: {DRIVE_OUTPUT}")

#

The "Continuity" Workflow1	Generate Clip 1	Establishes the

1.   List item
2.   List item

scene.
2	Extract Last Frame	Captures the exact pose/lighting at the end.
3	Upload Frame to "Image-to-Video"	Tells the AI exactly where to start Clip 2.
4	Describe the Next Action	The AI will animate the transition from that pos

In [13]:
import cv2
import os

# Put the name of your generated video file here
video_path = '/content/Wan2GP/outputs/video_001.mp4'
output_image = '/content/starting_frame_for_clip_2.jpg'

vidcap = cv2.VideoCapture(video_path)
total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Read the very last frame
vidcap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
success, image = vidcap.read()

if success:
    cv2.imwrite(output_image, image)
    print(f"✅ Success! Last frame saved as: {output_image}")
else:
    print("❌ Failed to extract frame. Check the video path.")

❌ Failed to extract frame. Check the video path.
